In [1]:
import pandas as pd
import numpy as np

## Retrieve data

In [2]:
etm_demand = pd.read_excel('Demand_Scenarios_TYNDP_2024_After_Public_Consultation.xlsb', sheet_name='3_DEMAND_OUTPUT', header=1)
etm_demand = etm_demand.copy()
etm_demand.drop(columns=["STUDY", "TYPE", "DASHBOARD_ID", "PARAMETER", "ENERGY_TYPE", "UNIT"], inplace=True)
print(etm_demand["SECTOR"].unique())
print(etm_demand["SUBSECTOR"].unique())
etm_demand_elc = etm_demand[(etm_demand['ENERGY_CARRIER'] == 'Electricity')
                            & (etm_demand['COUNTRY'] == 'IT')
                            ].drop(columns=['COUNTRY', 'ENERGY_CARRIER'])
etm_demand_elc = etm_demand_elc.reset_index(drop=True).set_index(['SECTOR', 'SUBSECTOR'])
display(etm_demand_elc)

['Households' 'Buildings' 'Industry' 'Transport' 'Agriculture' 'Other'
 'Energy']
['Total' 'Space heating' 'Cooling' 'Hot water' 'Appliances'
 'Space heating & hot water' 'Aluminium' 'Chemicals' 'Fertilizers' 'Food'
 'Metals' 'Others' 'Paper' 'Refineries' 'Steel' 'Cars' 'Busses' 'Trucks'
 'Vans' 'Passenger trains' 'Freight trains' 'Ships' 'Planes'
 'International aviation' 'International shipping' 'Datacenters']


2019        2040        2050  \
SECTOR      SUBSECTOR                                                       
Households  Total                       63.161167   90.173107   93.153698   
Buildings   Total                       82.318909   79.040302   65.689277   
Industry    Total                      136.440414  153.809828  172.140150   
            Total                        0.000000    0.000000    0.000000   
Transport   Total                        5.899005   60.101622   96.669509   
Agriculture Total                        5.839801    7.191769    8.495789   
Other       Total                        0.000000    0.000000    0.000000   
Energy      Total                        9.903151    9.903151    9.903151   
Households  Space heating                4.658522   24.687289   29.630553   
            Cooling                      7.985249    9.175052   10.202392   
            Hot water                    2.001705   11.423445   15.235789   
            Appliances                  48.515691   44.887321   38.084964   
Buildings   Space heating & hot water   19.971386   19.008871   11.277854   
            Cooling                     10.415924    8.338394    6.778557   
            Appliances                  51.931599   51.693037   47.632866   
Industry    Aluminium                    0.000000    0.000000    0.000000   
            Chemicals                   13.553087   13.994461   16.384499   
            Chemicals                    0.000000    0.000000    0.000000   
            Fertilizers                  0.108539    0.095738    0.085715   
            Fertilizers                  0.000000    0.000000    0.000000   
            Food                        13.456413   13.171264   14.050980   
            Metals                       2.035884    1.391113    1.160393   
            Others                      66.583945   78.677078   90.199084   
            Others                       0.000000    0.000000    0.000000   
            Paper                        8.698857    7.577085    7.937381   
            Refineries                   5.990478    2.032292    0.844781   
            Refineries                   0.000000    0.000000    0.000000   
            Steel                       18.978489   19.830790   20.446055   
Transport   Cars                         0.049358   39.533665   60.626461   
            Busses                       0.035973    2.272657    6.220444   
            Trucks                       0.000000    4.135605   10.346772   
            Vans                         0.008642    7.087481   11.118465   
            Passenger trains             4.683164    4.296037    4.147360   
            Freight trains               0.866688    0.714402    0.703314   
            Ships                        0.212593    0.199607    0.215556   
            Planes                       0.000000    1.453424    2.445696   
            Others                       0.042587    0.408745    0.845440   
            International aviation       0.000000    0.000000    0.000000   
            International shipping       0.000000    0.000000    0.000000   
Industry    Datacenters                  7.034722   17.040007   21.031262   

                                           2040.1      2050.1  
SECTOR      SUBSECTOR                                          
Households  Total                       83.960824   87.295047  
Buildings   Total                       77.821158   65.521541  
Industry    Total                      146.985378  158.373348  
            Total                        0.000000    0.000000  
Transport   Total                       50.243471   82.104564  
Agriculture Total                        6.005201    6.010874  
Other       Total                        0.000000    0.000000  
Energy      Total                        9.903151    9.903151  
Households  Space heating               22.018580   27.299294  
            Cooling                      9.399179   10.507316  
            Hot water                    8

## Postprocess data

In [9]:
# Total
etm_demand_elc_tot = etm_demand_elc[etm_demand_elc.index.get_level_values('SUBSECTOR') == 'Total']
total = etm_demand_elc_tot.sum().to_frame().rename(columns={0: 'Total'})

# Transport
etm_demand_elc_tra = etm_demand_elc[
    (etm_demand_elc.index.get_level_values('SECTOR') == 'Transport') &
    (etm_demand_elc.index.get_level_values('SUBSECTOR') == 'Total')
    ]
transport = etm_demand_elc_tra.sum().to_frame().rename(columns={0: 'Transport'})

# Households and Buildings: heating
heating_index = ['Space heating', 'Hot water', 'Space heating & hot water']
cooling_index = ['Cooling']
etm_demand_elc_heat = etm_demand_elc[
    ((etm_demand_elc.index.get_level_values('SECTOR') == 'Households') |
    (etm_demand_elc.index.get_level_values('SECTOR') == 'Buildings')) &
    (etm_demand_elc.index.get_level_values('SUBSECTOR').isin(heating_index))
    ]
heating = etm_demand_elc_heat.sum().to_frame().rename(columns={0: 'Heating'})

# Households and Buildings: cooling
etm_demand_elc_cool = etm_demand_elc[
    ((etm_demand_elc.index.get_level_values('SECTOR') == 'Households') |
    (etm_demand_elc.index.get_level_values('SECTOR') == 'Buildings')) &
    (etm_demand_elc.index.get_level_values('SUBSECTOR').isin(cooling_index))
    ]
cooling = etm_demand_elc_cool.sum().to_frame().rename(columns={0: 'Cooling'})

### All scenarios

In [25]:
df_abs = round(pd.concat([total,
                          transport,
                          heating,
                          cooling], axis=1).rename(
                              index = {2040: 'DS-2040',
                                       2050: 'DS-2050',
                                       '2040.1': 'GA-2040',
                                       '2050.1': 'GA-2050'
                                       }),0)
print("==================== Final elc consumption (TWh) ====================")
display(df_abs)

# DataFrame con gli share rispetto al totale (in percentuale)
print("==================== Final elc consumption - share over total (%) ====================")
df_share = round(df_abs.div(df_abs['Total'], axis=0) * 100, 0).drop(columns=['Total'])
display(df_share)

# DataFrame con gli aumenti percentuali rispetto al 2019
print("==================== Final elc consumption - growth over 2019 (%) ====================")
df_growth = round((df_abs / df_abs.loc[2019] - 1) * 100, 0).drop(index=[2019])
display(df_growth)

==================== Final elc consumption (TWh) ====================


,Total,Transport,Heating,Cooling
2019,304.0,6.0,27.0,18.0
DS-2040,400.0,60.0,55.0,18.0
DS-2050,446.0,97.0,56.0,17.0
GA-2040,375.0,50.0,48.0,18.0
GA-2050,409.0,82.0,51.0,17.0


==================== Final elc consumption - share over total (%) ====================


,Transport,Heating,Cooling
2019,2.0,9.0,6.0
DS-2040,15.0,14.0,4.0
DS-2050,22.0,13.0,4.0
GA-2040,13.0,13.0,5.0
GA-2050,20.0,12.0,4.0


==================== Final elc consumption - growth over 2019 (%) ====================


,Total,Transport,Heating,Cooling
DS-2040,32.0,900.0,104.0,0.0
DS-2050,47.0,1517.0,107.0,-6.0
GA-2040,23.0,733.0,78.0,0.0
GA-2050,35.0,1267.0,89.0,-6.0


### Average scenario

In [30]:
def df_avg(df, reference=True):
    if reference:
        df_avg = pd.DataFrame({
        2019: df.loc[2019],
        2040: df.loc[['DS-2040', 'GA-2040']].mean(),
        2050: df.loc[['DS-2050', 'GA-2050']].mean()
    }).T.round(0)
    else:
        df_avg = pd.DataFrame({
        2040: df.loc[['DS-2040', 'GA-2040']].mean(),
        2050: df.loc[['DS-2050', 'GA-2050']].mean()
    }).T.round(0)
    return df_avg

print("==================== Final elc consumption - average scenario (TWh) ====================")
display(df_avg(df_abs))

# Share rispetto al totale
print("==================== Final elc consumption - share over total (%) ====================")
display(df_avg(df_share))

# Crescita rispetto al 2019
print("==================== Final elc consumption - growth over 2019 (%) ====================")
display(df_avg(df_growth, reference=False))

==================== Final elc consumption - average scenario (TWh) ====================


,Total,Transport,Heating,Cooling
2019,304.0,6.0,27.0,18.0
2040,388.0,55.0,52.0,18.0
2050,428.0,90.0,54.0,17.0


==================== Final elc consumption - share over total (%) ====================


,Transport,Heating,Cooling
2019,2.0,9.0,6.0
2040,14.0,14.0,4.0
2050,21.0,12.0,4.0


==================== Final elc consumption - growth over 2019 (%) ====================


,Total,Transport,Heating,Cooling
2040,28.0,816.0,91.0,0.0
2050,41.0,1392.0,98.0,-6.0
